Code to get CI events based on Marquis et al. 202x, using FLEXTRKR data.

In [ ]:
import numpy as np
import xarray as xr
from pathlib import Path

In [3]:
path = Path('/neelin2020/ARM/CACTI')
fil = [str(i) for i in path.glob('celltrack_database*.nc')]
ds = xr.open_dataset(fil[0])
ds

<xarray.Dataset>
Dimensions:                    (tracks: 6892, times: 60, z: 45)
Coordinates:
  * tracks                     (tracks) int32 0 1 2 3 4 ... 6888 6889 6890 6891
    reltime                    int32 ...
  * times                      (times) int64 0 1 2 3 4 5 6 ... 54 55 56 57 58 59
  * z                          (z) float64 0.0 500.0 1e+03 ... 2.15e+04 2.2e+04
Data variables: (12/83)
    CAPE_mu                    (tracks) float32 ...
    CIN_IB_mu                  (tracks) float32 ...
    LCL_height_mu              (tracks) float32 ...
    LFC_height_mu              (tracks) float32 ...
    EL_height_mu               (tracks) float32 ...
    initial_ht_parcel_mu       (tracks) float32 ...
    ...                         ...
    max_zdr                    (tracks, times, z) float32 ...
    max_kdp                    (tracks, times, z) float32 ...
    max_rainrate               (tracks, times, z) float32 ...
    max_Dm                     (tracks, times, z) float32 ...
    max_lwc                    (tracks, times, z) float32 ...
    volrain                    (tracks, times, z) float32 ...
Attributes:
    title:                 CACTI C-SAPR2 cell tracking database
    Institution:           Pacific Northwest National Laboratoy
    Contact:               Zhe Feng, zhe.feng@pnnl.gov
    Created_on:            Tue Feb 22 09:34:55 2022
    startdate:             20181015.0000
    enddate:               20190303.0000
    time_resolution_hour:  0.25
    pixel_radius_km:       0.5

In [5]:
# get base time of each cell; this is the CI time
CI_time = ds.start_basetime

In [6]:
# read ARMBE
fils_ARMBE = [str(i) for i in path.glob('corarmbeatmM1*.nc')]
ds_ARMBE = xr.open_mfdataset(fils_ARMBE)

Filter ARMBE dataset such only timestamps with at least 2 hours before a CI event are recorded

In [10]:
# time difference between CI and ARMBE
nearest_ARMBE_time = (CI_time - ds_ARMBE.time)  

# filter for times where CI happens during/after radiosonde launch (CI time - ARMBE time >= 0)
cond = nearest_ARMBE_time.astype('int') >= 0  

# for each CI (dim = 'tracks'), select the closest ARMBE time
nearest_ARMBE_time = nearest_ARMBE_time.where(cond).min('tracks').dropna('time')  

# convert time difference from nanoseconds to hours
closest_CI_time = [np.timedelta64(i, 'ns').astype('timedelta64[s]').item().seconds/3600 for i in nearest_ARMBE_time.values] 

# store time difference in xarray Dataset
closest_CI_time = xr.DataArray(closest_CI_time, 
                               coords={'time': nearest_ARMBE_time.time}, dims=['time'])

# further filter on number of hours between radiosonde and CI, using 2 hours here for example
closest_CI_time = closest_CI_time.where(closest_CI_time < 2, drop=True)

# choose ARMBE subset such that only times closest to and before CI are selected
ds_ARMBE_subset = ds_ARMBE.sel(time = closest_CI_time.time)

In [11]:
# work with ds_ARMBE_subset
ds_ARMBE_subset

<xarray.Dataset>
Dimensions:                    (time: 48, bound: 2, pressure: 37, height: 512)
Coordinates:
  * time                       (time) datetime64[ns] 2018-09-23T11:30:00 ... ...
  * pressure                   (pressure) float32 1e+03 975.0 ... 125.0 100.0
  * height                     (height) float32 15.0 60.0 ... 2.301e+04
Dimensions without coordinates: bound
Data variables: (12/35)
    base_time                  (time) datetime64[ns] 2018-09-23 ... 2018-09-23
    time_offset                (time) datetime64[ns] dask.array<chunksize=(48,), meta=np.ndarray>
    time_bounds                (time, bound) datetime64[ns] dask.array<chunksize=(48, 2), meta=np.ndarray>
    pressure_bounds            (time, pressure, bound) float32 dask.array<chunksize=(48, 37, 2), meta=np.ndarray>
    height_bounds              (time, height, bound) float32 dask.array<chunksize=(48, 512, 2), meta=np.ndarray>
    height_10m                 (time) float32 10.0 10.0 10.0 ... 10.0 10.0 10.0
    ...                         ...
    omega_nwp_p                (time, pressure) float32 dask.array<chunksize=(48, 37), meta=np.ndarray>
    temperature_nwp_p          (time, pressure) float32 dask.array<chunksize=(48, 37), meta=np.ndarray>
    relative_humidity_nwp_p    (time, pressure) float32 dask.array<chunksize=(48, 37), meta=np.ndarray>
    lat                        (time) float32 -32.13 -32.13 ... -32.13 -32.13
    lon                        (time) float32 -64.73 -64.73 ... -64.73 -64.73
    alt                        (time) float32 1.141e+03 1.141e+03 ... 1.141e+03
Attributes: (12/17)
    Conventions:           ARM-1.2
    command_line:          idl -R -n armbeatm -s cor -f M1 -b 20180923 -e 201...
    title:                 ARM Best Estimate (ARMBE) Product, atmospheric pro...
    description:           ARM Best Estimate hourly averaged qc controlled pr...
    location_description:  Cloud, Aerosol, and Complex Terrain Interactions (...
    platform_id:           armbeatm
    ...                    ...
    data_level:            c1
    dod_version:           armbeatm-c1-1.8
    process_version:       vap-armbeatm-1.0-0.dev6.dirty.el6
    input_datastreams:     cor30qcecorM1.s1 : 1.2 : 20180923.000000-20190101....
    doi:                   10.5439/1333748
    history:               created by user zhang24 on machine lead at 2021-01...